Hack Environment Set Up
========================
Install the necessary client libraries in order to demo running arbitrary code on AI compute platform. Those steps are not needed once productionized. 

In [ ]:
%sh git clone -b prototype "https://github.com/ygong1/llm-foundry.git" ~/llm-foundry

In [ ]:
%sh cd ~/llm-foundry && pip install .

In [ ]:
dbutils.library.restartPython()

Submit Run Remotely On a Cluster
-------------------------------

Wrap up the config and submit the remote run.


In [4]:
from ygong.mosaic import ScalingConfig
from ygong.mosaic import submit
from ygong.mosaic import TrainingConfig
from ygong.mosaic.wsfs import WSFSIntegration

# TODO(ygong): add a spark example mention it is reading from UC table and dump to UC volume
parameters = { 
     "name": "custom-train-demo", 
     "seed": 42, 
     "device_train_microbatch_size": 8,
     "loggers": {
        "mlflow": {
            "tracking_uri": "databricks",
            "synchronous": False,
            "log_system_metrics": True
        }
     }
}
custom_code_repo_dir = "/Workspace/Users/yu.gong@databricks.com/.ide/custom-train-demo-4bf5c137"
# TODO(shitao): add a prioirty field with good default. We can show it in demo
config = TrainingConfig(
    name="custom-train-demo", 
    entry_point=f'{custom_code_repo_dir}/src/train.py', 
    parameters=parameters)

scalingConfig = ScalingConfig(gpusNum=8, poolName="r1z1")

# DEMO NOTE:
# This is temporary hack to mock the behavior of mounting workspace filesystem to the remote training
# nodes. Once the workspace filesystem fusion is integrated with netphos and dblet. We will get this for free.
wsfs = WSFSIntegration(wsfs_path=custom_code_repo_dir)

run = submit(config, scalingConfig, wait_job_to_finish=True, debug=True, wsfs=wsfs) 

# TODO(ygong): where is the code for mlflow (in launcher or args??)

,name,ID,User,Cluster,Nodes,Instance,Resumption,Status,Start Time,End Time,Experiment Run
0,custom-train-demo-hS6oeQ,831de17d-1ef6-4ff1-a74b-240b544d5d41,yu.gong@databricks.com,r1z1,1,8x a100_80gb,0,Completed,2024-03-28 11:27 PM,2024-03-28 11:34 PM,


In [ ]:
from mcli import RunStatus
if run.status == RunStatus.COMPLETED:
  print("Succeeded!")
else:
  raise Exception("Pretaining job faile")




Succeeded!
